In [10]:
import numpy
import pandas
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2


In [11]:
train_data_dir = 'D:/My Documents/GIT Projects/IEEE-Hack-the-meta/ASL_F/asl_alphabet_train'
test_data_dir = 'D:/My Documents/GIT Projects/IEEE-Hack-the-meta/ASL_F/asl_alphabet_test'

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=64, 
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical'
)

Found 82650 images belonging to 29 classes.
Found 4411 images belonging to 29 classes.


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(29))
model.add(Activation('softmax'))

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

In [7]:
steps_per_epoch = len(train_generator)
validation_steps = len(test_generator)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=test_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping],
)

Epoch 1/20
1292/1292 [==============================] - 846s 651ms/step - loss: 3.1096 - accuracy: 0.1011 - val_loss: 2.1996 - val_accuracy: 0.3555
Epoch 2/20
1292/1292 [==============================] - 685s 530ms/step - loss: 2.3396 - accuracy: 0.2682 - val_loss: 1.4736 - val_accuracy: 0.5493
Epoch 3/20
1292/1292 [==============================] - 473s 366ms/step - loss: 1.9543 - accuracy: 0.3610 - val_loss: 1.1740 - val_accuracy: 0.6416
Epoch 4/20
1292/1292 [==============================] - 458s 355ms/step - loss: 1.7212 - accuracy: 0.4266 - val_loss: 0.9676 - val_accuracy: 0.6905
Epoch 5/20
1292/1292 [==============================] - 392s 303ms/step - loss: 1.5451 - accuracy: 0.4742 - val_loss: 0.8511 - val_accuracy: 0.7096
Epoch 6/20
1292/1292 [==============================] - 388s 300ms/step - loss: 1.4098 - accuracy: 0.5120 - val_loss: 0.7060 - val_accuracy: 0.7611
Epoch 7/20
1292/1292 [==============================] - 385s 298ms/step - loss: 1.3140 - accuracy: 0.5454 - val_

In [8]:
score = model.evaluate(test_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.33901938796043396
Test accuracy: 0.8734980821609497


In [9]:
model.save('asl_model.h5')